In [ ]:
# Installing PYSPARK library:

pip install pyspark


In [5]:
import pyspark
import json

In [6]:
# Generating SPARK Session:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#Reading JSON file
df = spark.read.json("donation_np.json")
df.printSchema()     # Returns schema of table
df.show()            # Shows all records
df.count()           # Returns total count of rows

root
 |-- Address: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- Contribution Mode: string (nullable = true)
 |-- Financial Year: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- PAN Given: string (nullable = true)
 |-- Party: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- _corrupt_record: string (nullable = true)
 |-- field10: string (nullable = true)
 |-- field11: string (nullable = true)
 |-- field12: string (nullable = true)
 |-- field13: string (nullable = true)
 |-- field14: string (nullable = true)
 |-- field9: string (nullable = true)

+--------------------+--------+--------------------+--------------+--------------------+---------+------+------+---------------+-------+-------+-------+-------+-------+------+
|             Address|  Amount|   Contribution Mode|Financial Year|                Name|PAN Given| Party|  Type|_corrupt_record|field10|field11|field12|field13|field14|field9|
+--------------------+--------+---------

13680

In [7]:
# To rename the column : There is a function in spark : withColumnRenamed("existing_string","new_name")
data = df.withColumnRenamed("Contribution Mode","mode_of_payment").withColumnRenamed("Financial Year","fin_year").withColumnRenamed("PAN GIVEN","pan_given")
data.show()

+--------------------+--------+--------------------+--------+--------------------+---------+------+------+---------------+-------+-------+-------+-------+-------+------+
|             Address|  Amount|     mode_of_payment|fin_year|                Name|pan_given| Party|  Type|_corrupt_record|field10|field11|field12|field13|field14|field9|
+--------------------+--------+--------------------+--------+--------------------+---------+------+------+---------------+-------+-------+-------+-------+-------+------+
|                null|    null|                null|    null|                null|     null|  null|  null|              [|   null|   null|   null|   null|   null|  null|
|16-B, Ferozeshah ...| 3000000|                CASH| 2010-11|          Aziz Pasha|        Y|   CPI|Others|           null|       |       |       |       |       |      |
|No.1, First Floor...|10000000|000037, HDFC Bank...| 2014-15|    V K Ramachandran|        Y|CPI(M)|Others|           null|       |       |       |    

In [8]:
# [na] is for null values and drop is used to delete the particular subset
# That means following query or command will delete all rows which has name as null value

data = data.na.drop(subset=["Name"])
data = data.drop("field10", "field11", "field12", "field13", "field14", "field9")
data.show()
data.printSchema()

+--------------------+--------+--------------------+--------+--------------------+---------+------+------+---------------+
|             Address|  Amount|     mode_of_payment|fin_year|                Name|pan_given| Party|  Type|_corrupt_record|
+--------------------+--------+--------------------+--------+--------------------+---------+------+------+---------------+
|16-B, Ferozeshah ...| 3000000|                CASH| 2010-11|          Aziz Pasha|        Y|   CPI|Others|           null|
|No.1, First Floor...|10000000|000037, HDFC Bank...| 2014-15|    V K Ramachandran|        Y|CPI(M)|Others|           null|
|3, Motilal Nehru ...|  108000|Cheque, State Ban...| 2014-15|  Dr. Manmohan Singh|        N|   INC|Others|           null|
|9,Firozshah Road ...|   54000|Through Bank Tran...| 2011-12| Dr. Manda Jagnathan|        N|   INC|Others|           null|
|17,Dr.B.R.Mehta L...|   54000|Through Bank Tran...| 2011-12|    Prof. K.V.Thomas|        N|   INC|Others|           null|
|31 Shamla Hills

In [9]:
# Encrypting particular column :

from pyspark.sql.functions import sha2, concat_ws,col
data = data.withColumn("Address", sha2(concat_ws("||", data.Address),256))
data.show()

+--------------------+--------+--------------------+--------+--------------------+---------+------+------+---------------+
|             Address|  Amount|     mode_of_payment|fin_year|                Name|pan_given| Party|  Type|_corrupt_record|
+--------------------+--------+--------------------+--------+--------------------+---------+------+------+---------------+
|5a3058deb6f337958...| 3000000|                CASH| 2010-11|          Aziz Pasha|        Y|   CPI|Others|           null|
|846539cb21bc9e6c6...|10000000|000037, HDFC Bank...| 2014-15|    V K Ramachandran|        Y|CPI(M)|Others|           null|
|869fa3a19f1c51ad1...|  108000|Cheque, State Ban...| 2014-15|  Dr. Manmohan Singh|        N|   INC|Others|           null|
|5f04f40130569ddab...|   54000|Through Bank Tran...| 2011-12| Dr. Manda Jagnathan|        N|   INC|Others|           null|
|524b1379d08e4c02f...|   54000|Through Bank Tran...| 2011-12|    Prof. K.V.Thomas|        N|   INC|Others|           null|
|6db7af0c5dca3b3

In [10]:
# Categorizing mode of payment in different modes [ rough ]:
# We can improve it by applying REGEX n all

from pyspark.sql.functions import when

data = data.withColumn("mode_of_payment", \
    when((data.mode_of_payment.contains("CASH")), "Cash") \
   .when((data.mode_of_payment.contains("Cheque")),"Cheque")\
   .when((data.mode_of_payment.contains("Bank")),"Bank")\
   .otherwise("Others")\
  )
  
data.show()

+--------------------+--------+---------------+--------+--------------------+---------+------+------+---------------+
|             Address|  Amount|mode_of_payment|fin_year|                Name|pan_given| Party|  Type|_corrupt_record|
+--------------------+--------+---------------+--------+--------------------+---------+------+------+---------------+
|5a3058deb6f337958...| 3000000|           Cash| 2010-11|          Aziz Pasha|        Y|   CPI|Others|           null|
|846539cb21bc9e6c6...|10000000|           Bank| 2014-15|    V K Ramachandran|        Y|CPI(M)|Others|           null|
|869fa3a19f1c51ad1...|  108000|         Cheque| 2014-15|  Dr. Manmohan Singh|        N|   INC|Others|           null|
|5f04f40130569ddab...|   54000|           Bank| 2011-12| Dr. Manda Jagnathan|        N|   INC|Others|           null|
|524b1379d08e4c02f...|   54000|           Bank| 2011-12|    Prof. K.V.Thomas|        N|   INC|Others|           null|
|6db7af0c5dca3b333...|  100000|         Others| 2011-12|

In [26]:
#Performing aggregation functions:
from pyspark.sql.types import IntegerType

data = data.withColumn("Amount",col("Amount").cast(IntegerType()))
tmp = data.groupBy('Party').sum("Amount")
cnt = data.groupBy('Party').count()
avgv = data.groupBy('Party').avg("Amount")
mx = data.groupBy('Party').max("Amount")

inc_top_donor = data.select("Name").filter(data.Amount == mx.collect()[0][1])
bjp_top_donor = data.select("Name").filter(data.Amount == mx.collect()[1][1])
ncp_top_donor = data.select("Name").filter(data.Amount == mx.collect()[2][1])
cpi_top_donor = data.select("Name").filter(data.Amount == mx.collect()[3][1])
cpim_top_donor = data.select("Name").filter(data.Amount == mx.collect()[4][1])

tmp.show()
cnt.show()
avgv.show()
mx.show()
cpi_top_donor.show()
cpim_top_donor.show()
inc_top_donor.show()
bjp_top_donor.show()

+------+-----------+
| Party|sum(Amount)|
+------+-----------+
|   INC| 4031487349|
|   BJP| 9295525996|
|   NCP|  647911419|
|   CPI|   68123698|
|CPI(M)|  150622128|
+------+-----------+

+------+-----+
| Party|count|
+------+-----+
|   INC| 3785|
|   BJP| 8782|
|   NCP|  107|
|   CPI|  384|
|CPI(M)|  515|
+------+-----+

+------+------------------+
| Party|       avg(Amount)|
+------+------------------+
|   INC|1065122.1529722589|
|   BJP|1058474.8344340697|
|   NCP| 6055246.906542056|
|   CPI|177405.46354166666|
|CPI(M)| 292470.1514563107|
+------+------------------+

+------+-----------+
| Party|max(Amount)|
+------+-----------+
|   INC|  500000000|
|   BJP|  500000000|
|   NCP|   50000000|
|   CPI|    3000000|
|CPI(M)|   10000000|
+------+-----------+

+--------------------+
|                Name|
+--------------------+
|          Aziz Pasha|
|Arabindo Pharma Ltd.|
|Finolex Industrie...|
|Iskrupa Mall Mana...|
|Janak Dinkarrai D...|
| Lodha Constructions|
|Lodha Dwellers Pv...|
|

In [23]:
# # Aggregate for SUM: [ Manually]
# data = data.withColumn("INC_SUM_LTD", \
#     when((data.Party == 'INC'), tmp.collect()[0][1]).otherwise(0))

# data = data.withColumn("BJP_SUM_LTD", \
#     when((data.Party == 'BJP'), tmp.collect()[1][1]).otherwise(0))

# data = data.withColumn("NCP_SUM_LTD", \
#     when((data.Party == 'NCP'), tmp.collect()[2][1]).otherwise(0))

# data = data.withColumn("CPI_SUM_LTD", \
#     when((data.Party == 'CPI'), tmp.collect()[3][1]).otherwise(0))

# data = data.withColumn("CPI(M)_SUM_LTD", \
#     when((data.Party == 'CPI(M)'), tmp.collect()[4][1]).otherwise(0))

# # Aggregate for SUM: [ Using function]
party_sum = ("INC", "BJP", "NCP", "CPI", "CPI(M)")
for i,party in enumerate(party_sum):
    data = data.withColumn("{}_SUM_LTD".format(party), \
    when((data.Party == party), tmp.collect()[i][1]).otherwise(0))


data.select("Party","CPI_SUM_LTD","CPI(M)_SUM_LTD","INC_SUM_LTD","BJP_SUM_LTD", "NCP_SUM_LTD").show(1000)

+------+-----------+--------------+-----------+-----------+-----------+
| Party|CPI_SUM_LTD|CPI(M)_SUM_LTD|INC_SUM_LTD|BJP_SUM_LTD|NCP_SUM_LTD|
+------+-----------+--------------+-----------+-----------+-----------+
|   CPI|   68123698|             0|          0|          0|          0|
|CPI(M)|          0|     150622128|          0|          0|          0|
|   INC|          0|             0| 4031487349|          0|          0|
|   INC|          0|             0| 4031487349|          0|          0|
|   INC|          0|             0| 4031487349|          0|          0|
|   BJP|          0|             0|          0| 9295525996|          0|
|   BJP|          0|             0|          0| 9295525996|          0|
|   BJP|          0|             0|          0| 9295525996|          0|
|   BJP|          0|             0|          0| 9295525996|          0|
|   INC|          0|             0| 4031487349|          0|          0|
|   BJP|          0|             0|          0| 9295525996|     

In [27]:
# Aggregate for Count: [Manually]
# data = data.withColumn("INC_COUNT_LTD", \
#     when((data.Party == 'INC'), cnt.collect()[0][1]).otherwise(0))

# data = data.withColumn("BJP_COUNT_LTD", \
#     when((data.Party == 'BJP'), cnt.collect()[1][1]).otherwise(0))

# data = data.withColumn("NCP_COUNT_LTD", \
#     when((data.Party == 'NCP'), cnt.collect()[2][1]).otherwise(0))

# data = data.withColumn("CPI_COUNT_LTD", \
#     when((data.Party == 'CPI'), cnt.collect()[3][1]).otherwise(0))

# data = data.withColumn("CPI(M)_COUNT_LTD", \
#     when((data.Party == 'CPI(M)'), cnt.collect()[4][1]).otherwise(0))

# Aggregate for Count: [Using Function]
for i,party in enumerate(party_sum):
    data = data.withColumn("{}_COUNT_LTD".format(party), \
    when((data.Party == party), cnt.collect()[i][1]).otherwise(0))


data.select("Party","CPI_COUNT_LTD","CPI(M)_COUNT_LTD","INC_COUNT_LTD","BJP_COUNT_LTD","NCP_COUNT_LTD").show()

+------+-------------+----------------+-------------+-------------+-------------+
| Party|CPI_COUNT_LTD|CPI(M)_COUNT_LTD|INC_COUNT_LTD|BJP_COUNT_LTD|NCP_COUNT_LTD|
+------+-------------+----------------+-------------+-------------+-------------+
|   CPI|          384|               0|            0|            0|            0|
|CPI(M)|            0|             515|            0|            0|            0|
|   INC|            0|               0|         3785|            0|            0|
|   INC|            0|               0|         3785|            0|            0|
|   INC|            0|               0|         3785|            0|            0|
|   BJP|            0|               0|            0|         8782|            0|
|   BJP|            0|               0|            0|         8782|            0|
|   BJP|            0|               0|            0|         8782|            0|
|   BJP|            0|               0|            0|         8782|            0|
|   INC|        

In [28]:
# Aggregate for Average:
data = data.withColumn("INC_AVG_LTD", \
    when((data.Party == 'INC'), avgv.collect()[0][1]).otherwise(0))

data = data.withColumn("BJP_AVG_LTD", \
    when((data.Party == 'BJP'), avgv.collect()[1][1]).otherwise(0))

data = data.withColumn("NCP_AVG_LTD", \
    when((data.Party == 'NCP'), avgv.collect()[2][1]).otherwise(0))

data = data.withColumn("CPI_AVG_LTD", \
    when((data.Party == 'CPI'), avgv.collect()[3][1]).otherwise(0))

data = data.withColumn("CPI(M)_AVG_LTD", \
    when((data.Party == 'CPI(M)'), avgv.collect()[4][1]).otherwise(0))

# Aggregate for Average: [Using Function]
for i,party in enumerate(party_sum):
    data = data.withColumn("{}_AVG_LTD".format(party), \
    when((data.Party == party), avgv.collect()[i][1]).otherwise(0))



data.select("Party","CPI_AVG_LTD","CPI(M)_AVG_LTD","INC_AVG_LTD","BJP_AVG_LTD","NCP_AVG_LTD").show()

+------+------------------+-----------------+------------------+------------------+-----------+
| Party|       CPI_AVG_LTD|   CPI(M)_AVG_LTD|       INC_AVG_LTD|       BJP_AVG_LTD|NCP_AVG_LTD|
+------+------------------+-----------------+------------------+------------------+-----------+
|   CPI|177405.46354166666|              0.0|               0.0|               0.0|        0.0|
|CPI(M)|               0.0|292470.1514563107|               0.0|               0.0|        0.0|
|   INC|               0.0|              0.0|1065122.1529722589|               0.0|        0.0|
|   INC|               0.0|              0.0|1065122.1529722589|               0.0|        0.0|
|   INC|               0.0|              0.0|1065122.1529722589|               0.0|        0.0|
|   BJP|               0.0|              0.0|               0.0|1058474.8344340697|        0.0|
|   BJP|               0.0|              0.0|               0.0|1058474.8344340697|        0.0|
|   BJP|               0.0|             

In [29]:
# Aggregate for MAX:[ Manually]
data = data.withColumn("INC_MAX_LTD", \
    when((data.Party == 'INC'), mx.collect()[0][1]).otherwise(0))

data = data.withColumn("BJP_MAX_LTD", \
    when((data.Party == 'BJP'), mx.collect()[1][1]).otherwise(0))

data = data.withColumn("NCP_MAX_LTD", \
    when((data.Party == 'NCP'), mx.collect()[2][1]).otherwise(0))

data = data.withColumn("CPI_MAX_LTD", \
    when((data.Party == 'CPI'), mx.collect()[3][1]).otherwise(0))

data = data.withColumn("CPI(M)_MAX_LTD", \
    when((data.Party == 'CPI(M)'), mx.collect()[4][1]).otherwise(0))

# Aggregate for MAX:[ Using Function ]
for i,party in enumerate(party_sum):
    data = data.withColumn("{}_MAX_LTD".format(party), \
    when((data.Party == party), mx.collect()[i][1]).otherwise(0))


data.select("Party","CPI_MAX_LTD","CPI(M)_MAX_LTD","INC_MAX_LTD","BJP_MAX_LTD","NCP_MAX_LTD").show(1000)


+------+-----------+--------------+-----------+-----------+-----------+
| Party|CPI_MAX_LTD|CPI(M)_MAX_LTD|INC_MAX_LTD|BJP_MAX_LTD|NCP_MAX_LTD|
+------+-----------+--------------+-----------+-----------+-----------+
|   CPI|    3000000|             0|          0|          0|          0|
|CPI(M)|          0|      10000000|          0|          0|          0|
|   INC|          0|             0|  500000000|          0|          0|
|   INC|          0|             0|  500000000|          0|          0|
|   INC|          0|             0|  500000000|          0|          0|
|   BJP|          0|             0|          0|  500000000|          0|
|   BJP|          0|             0|          0|  500000000|          0|
|   BJP|          0|             0|          0|  500000000|          0|
|   BJP|          0|             0|          0|  500000000|          0|
|   INC|          0|             0|  500000000|          0|          0|
|   BJP|          0|             0|          0|  500000000|     

In [31]:
# TOP DONORS: 1] One by One (Manually):
data = data.withColumn("INC_TOP_DONOR", \
    when((data.Party == 'INC'), inc_top_donor.collect()[0][0]).otherwise('-'))

data = data.withColumn("BJP_TOP_DONOR", \
    when((data.Party == 'BJP'), bjp_top_donor.collect()[0][0]).otherwise('-'))

data = data.withColumn("NCP_TOP_DONOR", \
    when((data.Party == 'NCP'), ncp_top_donor.collect()[0][0]).otherwise('-'))

data = data.withColumn("CPI_TOP_DONOR", \
    when((data.Party == 'CPI'), cpi_top_donor.collect()[0][0]).otherwise('-'))

data = data.withColumn("CPIM_TOP_DONOR", \
    when((data.Party == 'CPI(M)'), cpim_top_donor.collect()[0][0]).otherwise('-'))

data.select("CPI_TOP_DONOR","CPIM_TOP_DONOR","INC_TOP_DONOR","BJP_TOP_DONOR","NCP_TOP_DONOR").show(1000)

+-------------+----------------+--------------------+--------------------+--------------------+
|CPI_TOP_DONOR|  CPIM_TOP_DONOR|       INC_TOP_DONOR|       BJP_TOP_DONOR|       NCP_TOP_DONOR|
+-------------+----------------+--------------------+--------------------+--------------------+
|   Aziz Pasha|               -|                   -|                   -|                   -|
|            -|V K Ramachandran|                   -|                   -|                   -|
|            -|               -|General Electoral...|                   -|                   -|
|            -|               -|General Electoral...|                   -|                   -|
|            -|               -|General Electoral...|                   -|                   -|
|            -|               -|                   -|General Electoral...|                   -|
|            -|               -|                   -|General Electoral...|                   -|
|            -|               -|        

In [32]:
#Applying GROUP BY to group the data firstly on Financial Year basis and then on Party name basis :
party_fin_year = data.groupBy('fin_year','Party').sum("Amount")
party_fin_year.show()
party_fin_year.count()

+--------+------+-----------+
|fin_year| Party|sum(Amount)|
+--------+------+-----------+
| 2008-09|   INC|  279018460|
| 2014-15|   BJP| 4373506898|
| 2014-15|   INC| 1414610950|
| 2014-15|   CPI|   13346675|
| 2010-11|CPI(M)|   15357072|
| 2010-11|   NCP|    1355000|
| 2012-13|   BJP|  831924462|
| 2013-14|   NCP|  140200000|
| 2009-10|CPI(M)|    3962049|
| 2003-04|CPI(M)|     200000|
| 2006-07|CPI(M)|    1124719|
| 2010-11|   INC|   80205884|
| 2014-15|CPI(M)|   34197498|
| 2010-11|   BJP|  146253279|
| 2004-05|   INC|  320555643|
| 2003-04|   CPI|     779148|
| 2005-06|   BJP|   36156111|
| 2011-12|CPI(M)|   23838657|
| 2004-05|   BJP|  339521289|
| 2004-05|CPI(M)|     896355|
+--------+------+-----------+
only showing top 20 rows



56

In [33]:
# Generating FINANCIAL YEAR - PARTY -TOTAL SUM REPORT:
"""
Also we should return the created dataframe with updated column and store into main dataframe as
it didn't get upadated inside function, only last iterable gets updated so thats why we have to return 
each dataframe with updated column
"""
from pyspark.sql.functions import concat,col,lit
data                    # We have to declare dataframe as global as inside function it gets LOCAL
def fin_party_sum(fin_year_part, party_name, sum_fin_year_party,data):
    return data.withColumn("{}_{}_SUM".format(fin_year_part,party_name),when( (data.Party == "{}".format(party_name)) & (data.fin_year == "{}".format(fin_year_part)) ,lit(sum_fin_year_party) ).otherwise(lit(0)))

for i in party_fin_year.collect():
    data = fin_party_sum(i[0], i[1], i[2],data)

data.columns

#Another Method for genrating Financial Year - Party Fund Report:
# from pyspark.sql.functions import concat,col,lit
# for fin_year_party, party_name, sum_fin_party in party_fin_year.select("fin_year", "Party", "sum(Amount)").collect():
#       data = data.withColumn("{}_{}_SUM".format(fin_year_party, party_name),when( (data.Party == "{}".format(party_name)) & (data.fin_year == "{}".format(fin_year_party)) ,lit(sum_fin_party) ).otherwise(lit(0)))

# data.columns


['Address',
 'Amount',
 'mode_of_payment',
 'fin_year',
 'Name',
 'pan_given',
 'Party',
 'Type',
 '_corrupt_record',
 'INC_SUM_LTD',
 'BJP_SUM_LTD',
 'NCP_SUM_LTD',
 'CPI_SUM_LTD',
 'CPI(M)_SUM_LTD',
 'INC_COUNT_LTD',
 'BJP_COUNT_LTD',
 'NCP_COUNT_LTD',
 'CPI_COUNT_LTD',
 'CPI(M)_COUNT_LTD',
 'INC_AVG_LTD',
 'BJP_AVG_LTD',
 'NCP_AVG_LTD',
 'CPI_AVG_LTD',
 'CPI(M)_AVG_LTD',
 'INC_MAX_LTD',
 'BJP_MAX_LTD',
 'NCP_MAX_LTD',
 'CPI_MAX_LTD',
 'CPI(M)_MAX_LTD',
 'INC_TOP_DONOR',
 'BJP_TOP_DONOR',
 'NCP_TOP_DONOR',
 'CPI_TOP_DONOR',
 'CPIM_TOP_DONOR',
 '2008-09_INC_SUM',
 '2014-15_BJP_SUM',
 '2014-15_INC_SUM',
 '2014-15_CPI_SUM',
 '2010-11_CPI(M)_SUM',
 '2010-11_NCP_SUM',
 '2012-13_BJP_SUM',
 '2013-14_NCP_SUM',
 '2009-10_CPI(M)_SUM',
 '2003-04_CPI(M)_SUM',
 '2006-07_CPI(M)_SUM',
 '2010-11_INC_SUM',
 '2014-15_CPI(M)_SUM',
 '2010-11_BJP_SUM',
 '2004-05_INC_SUM',
 '2003-04_CPI_SUM',
 '2005-06_BJP_SUM',
 '2011-12_CPI(M)_SUM',
 '2004-05_BJP_SUM',
 '2004-05_CPI(M)_SUM',
 '2013-14_CPI(M)_SUM',
 '200

In [34]:
# Creating dataframe for "MODE OF PAYMENT":
group_mode_of_payment = data.groupBy("mode_of_payment").count()
group_mode_of_payment.show()

+---------------+-----+
|mode_of_payment|count|
+---------------+-----+
|           Bank| 7281|
|         Cheque| 1993|
|           Cash|   29|
|         Others| 4270|
+---------------+-----+



In [35]:
#Generating " Count per MODE OF PAYMENT "" Report:
for payment_mode, cnt in group_mode_of_payment.select("mode_of_payment", "count").collect():
      data = data.withColumn("{}_count_LTD".format(payment_mode),when((data.mode_of_payment == "{}".format(payment_mode)),lit(cnt) ).otherwise(lit(0)))

data.select("Bank_count_LTD", "Cheque_count_LTD", "Cash_count_LTD", "Others_count_LTD").show()

+--------------+----------------+--------------+----------------+
|Bank_count_LTD|Cheque_count_LTD|Cash_count_LTD|Others_count_LTD|
+--------------+----------------+--------------+----------------+
|             0|               0|            29|               0|
|          7281|               0|             0|               0|
|             0|            1993|             0|               0|
|          7281|               0|             0|               0|
|          7281|               0|             0|               0|
|             0|               0|             0|            4270|
|             0|               0|             0|            4270|
|          7281|               0|             0|               0|
|             0|               0|             0|            4270|
|          7281|               0|             0|               0|
|          7281|               0|             0|               0|
|             0|               0|             0|            4270|
|         